<a href="https://colab.research.google.com/github/kerykai/face_recognition_for_colab/blob/master/trainer_photo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scipy==1.1.0

In [ ]:
%tensorflow_version 1.14
import tensorflow as tf
tf.__version__

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


'1.15.2'

In [ ]:
# 連接github
!pip install -q xlrd
!git clone https://github.com/kerykai/face_recognition_for_colab.git

fatal: destination path 'face_recognition_for_colab' already exists and is not an empty directory.


In [ ]:
# 連接google_drive
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

os.chdir("face_recognition_for_colab/")

In [ ]:
import os
import pickle
from sklearn.svm import LinearSVC

from packages import load_MTCNN as lmtcnn
from packages import load_face_pkl as pkl


In [ ]:
DATA_PATH = 'data/' #data目錄
    
IMG_OUT_PATH = os.path.join(DATA_PATH, "dataset") #image目錄

# FACENET_DATA_PATH = os.path.join(DATA_PATH, "facenet","20180402-114759","20180402-114759.pb") #dacenet路徑

SVM_DATA_PATH = os.path.join(DATA_PATH, "serializer", "lfw_svm_classifier.pkl") #svm路徑

COLAB_PATH = '/content/gdrive/My Drive/Colab Notebooks'
   
    
#----------載入MTCNN-----------#
    
emb_array, labels, labels_dict = lmtcnn.load_MTCNN(DATA_PATH)

#--------------保存facenet.pkl-------------#          
# 人臉特徵
emb_features_file = open(os.path.join(COLAB_PATH + '/recognizer', 'lfw_emb_features.pkl'), 'wb')
pickle.dump(emb_array, emb_features_file)
emb_features_file.close()

# 矩陣
emb_lables_file = open(os.path.join(COLAB_PATH + '/recognizer', 'lfw_emb_labels.pkl'), 'wb')
pickle.dump(labels, emb_lables_file)
emb_lables_file.close()

# user_ids
emb_lables_dict_file = open(os.path.join(COLAB_PATH + '/recognizer', 'lfw_emb_labels_dict.pkl'), 'wb')
pickle.dump(labels_dict, emb_lables_dict_file)
emb_lables_dict_file.close()
    
#-------------------準備相關變數-----------------#
    
# 訓練/測試變數
X_train = []; y_train = []
X_test = []; y_test = []

# 保存己經有處理過的人臉label
processed = set()
    
path = COLAB_PATH + '/recognizer'
    
emb_features, emb_labels, emb_labels_dict, emb_dict = pkl.load_face_pkl(path, COLAB_PATH)

# 分割訓練資料集與驗證資料集
for (emb_feature, emb_label) in zip(emb_features, emb_labels):
    if emb_label in processed:
        X_train.append(emb_feature)
        y_train.append(emb_label)
    else:
        X_test.append(emb_feature)
        y_test.append(emb_label)
        processed.add(emb_label)

# 結果
print('X_train: {}, y_train: {}'.format(len(X_train), len(y_train)))
print('X_test: {}, y_test: {}'.format(len(X_test), len(y_test)))


#----------------訓練人臉分類器(SVM Classifier)-----------------#
#使用scikit-learn的SVM分類器來進行訓練。
#使用linearSvc來訓練

# 訓練分類器
print('Training classifier')
linearsvc_classifier = LinearSVC(C=1, multi_class='crammer_singer')

# 進行訓練
linearsvc_classifier.fit(X_train, y_train)

classifier_filename = SVM_DATA_PATH


class_names = []
for key in sorted(emb_labels_dict.keys()):
    class_names.append(emb_labels_dict[key].replace('_', ' '))


with open(classifier_filename, 'wb') as outfile:
    pickle.dump((linearsvc_classifier, class_names), outfile)

print('Saved classifier model to file "%s"' % classifier_filename)

face_id_name = ''


Origin: Number of classes: 25
Origin: Number of images: 344
Model filename: /content/gdrive/My Drive/Colab Notebooks/facenet/20180402-114759/20180402-114759.pb
Instructions for updating:
Use tf.gfile.GFile.



100%|██████████| 688/688 [00:23<00:00, 29.26it/s] 


人臉特徵數量: 344, shape: (344, 512), type: <class 'numpy.ndarray'>
人臉標籤數量: 344, type: <class 'list'>
人臉標籤種類: {}, type: {} 25 <class 'dict'>
X_train: 319, y_train: 319
X_test: 25, y_test: 25
Training classifier
Saved classifier model to file "data/serializer/lfw_svm_classifier.pkl"


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
